In [1]:
import pdfplumber
import pandas as pd
import re
import glob
import string
import random

In [32]:
def extractor(path):   
    
    with pdfplumber.open(path) as pdf:
        page = pdf.pages[0]
        text = page.extract_text()
    try:    
        client_re = re.compile(r'CL\d{5}')
        ref_client = client_re.search(text)[0]
    except:
        ref_client = client_gen()
    fact_re = re.compile(r'FA\d{8}')
    fact_num = fact_re.search(text)[0]
    date_re = re.compile(r'\d{2}[/]\d{2}[/]\d{4}')
    date = date_re.search(text)[0]
    ref = []
    for row in text.split('\n'):
            if row.startswith('AR'):
                ref.append(row)
    df = pd.DataFrame(columns=['ref_client','fact_num','code_article','desc','qtt','pu','date'])
    for row in enumerate(ref):

        valeur = re.compile(r"\s\d.*")
        code_r = re.compile(r"^AR\d{5}")
        desc_r =re.compile(r"\s\D.*\D\s")

        qtt = valeur.search(ref[0])[0].split()[0]
        pu = valeur.search(ref[0])[0].split()[1]
        desc = desc_r.search(ref[0])[0]
        code = code_r.search(ref[0])[0]

        df = df.append({
            'ref_client': ref_client,
            'fact_num': fact_num,
            'code_article': code,
            'desc': desc,
            'qtt': qtt,
            'pu': pu,
            'date': date
            }, ignore_index=True)
    return df

def client_gen():
    size = 3
    num = string.digits
    client_num = "CL42"+"".join(random.sample(num,size))
    return client_num

In [33]:
def client_gen():
    size = 3
    num = string.digits
    client_num = "CL42"+"".join(random.sample(num,size))
    return client_num

In [34]:
path = "Factures_all/*"
list_path = []
for path in glob.glob(path):
    list_path.append(path)
list_path[:5]    

['Factures_all\\02600 Villers  Cotterets- Ma Boutique Tany- juin 2020- Facture 2786.pdf',
 'Factures_all\\04300 Forcalquier- Petra Patrimonia- Marjorie Thiers- mai 2020- Facture 2727.pdf',
 'Factures_all\\05000 Gap- AS- fevrier 2020- Facture 2679.pdf',
 'Factures_all\\05290 Vallouise- Les Vaudois- juin 2020- Facture 2789.pdf',
 'Factures_all\\06370 Mouans Sartoux- Boomerang- janvier 2020- Facture 2661.pdf']

In [35]:
facture = [] # the list with all invoice
facture_error = []
for path in list_path:
    try:
        facture.append(extractor(path))
        print('conversion OK:',path)
    except (ValueError):
        facture_error.append(path)
        print('conversion ERROR',path)
print("Nombre d'erreurs:",len(facture_error))
print("Nombre total de factures converties :",len(facture))

conversion OK: Factures_all\02600 Villers  Cotterets- Ma Boutique Tany- juin 2020- Facture 2786.pdf
conversion OK: Factures_all\04300 Forcalquier- Petra Patrimonia- Marjorie Thiers- mai 2020- Facture 2727.pdf
conversion OK: Factures_all\05000 Gap- AS- fevrier 2020- Facture 2679.pdf
conversion OK: Factures_all\05290 Vallouise- Les Vaudois- juin 2020- Facture 2789.pdf
conversion OK: Factures_all\06370 Mouans Sartoux- Boomerang- janvier 2020- Facture 2661.pdf
conversion OK: Factures_all\06600 Antibes- Herbularium la Melisse- octobre 2019- Facture 2567.pdf
conversion OK: Factures_all\08000 Charleville Mezieres- La Malinerie- mars 2020- Facture 2718.pdf
conversion OK: Factures_all\08000 Charleville Mezieres- La Malinerie- novembre 2020- Facture 2921.pdf
conversion OK: Factures_all\11000 Carcassonne- AS- fevrier 2020- Facture 2697.pdf
conversion OK: Factures_all\11000 Carcassonne- La Ferme- fevrier 2020- Facture 2698.pdf
conversion OK: Factures_all\11000 Carcassonne- La Ferme- octobre 2020- 

In [36]:
df = pd.concat(facture)

In [37]:
df

,ref_client,fact_num,code_article,desc,qtt,pu,date
0,CL00222,FA00002786,AR00100,SAS CITRON BLEU GM,"5,00","12,50",16/06/2020
1,CL00222,FA00002786,AR00100,SAS CITRON BLEU GM,"5,00","12,50",16/06/2020
2,CL00222,FA00002786,AR00100,SAS CITRON BLEU GM,"5,00","12,50",16/06/2020
3,CL00222,FA00002786,AR00100,SAS CITRON BLEU GM,"5,00","12,50",16/06/2020
4,CL00222,FA00002786,AR00100,SAS CITRON BLEU GM,"5,00","12,50",16/06/2020
...,...,...,...,...,...,...,...
3,CL42269,FA00002777,AR00084,SAS FEUILLAGE Vert GM,"5,00","12,50",11/06/2020
4,CL42269,FA00002777,AR00084,SAS FEUILLAGE Vert GM,"5,00","12,50",11/06/2020
5,CL42269,FA00002777,AR00084,SAS FEUILLAGE Vert GM,"5,00","12,50",11/06/2020
6,CL42269,FA00002777,AR00084,SAS FEUILLAGE Vert GM,"5,00","12,50",11/06/2020
